Loading the SQL Extension

In [1]:
%load_ext sql

Installing pandas and importing csv,sqlite.

In [2]:
import csv, sqlite3

con=sqlite3.connect("FinalDB.db")
cur=con.cursor()
!pip install -q pandas==1.1.5

Connecting to the FINALDB.db

In [3]:
%sql sqlite:///FinalDB.db

'Connected: @FinalDB.db'

Transferring the datasets to tables

In [4]:
import pandas

df=pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv")

df.to_sql("CENSUS_DATA",con,if_exists='replace',index=False,method="multi")

df=pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv")

df.to_sql("CHICAGO_CRIME_DATA",con,if_exists='replace',index=False,method="multi")

df=pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv")

df.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA",con,if_exists='replace',index=False,method="multi")

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  both result in 0.1234 being formatted as 0.12.


Question 1: How many crimes are in the Crime table 

In [5]:
%sql select COUNT(*) FROM CHICAGO_CRIME_DATA;

 * sqlite:///FinalDB.db
Done.


COUNT(*)
533


QUESTION 2: Listing which Community Areas have a Per Capita Income that's less than $11,000

In [16]:
%sql select COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PER_CAPITA_INCOME FROM CENSUS_DATA \
WHERE PER_CAPITA_INCOME <11000;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
26.0,West Garfield Park,10934
30.0,South Lawndale,10402
37.0,Fuller Park,10432
54.0,Riverdale,8201


Question#3: Listing the Case Numbers for Offenses involving Children

In [17]:
%sql select CASE_NUMBER,PRIMARY_TYPE FROM CHICAGO_CRIME_DATA \
WHERE PRIMARY_TYPE="OFFENSE INVOLVING CHILDREN"

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,PRIMARY_TYPE
HN567387,OFFENSE INVOLVING CHILDREN
HR391350,OFFENSE INVOLVING CHILDREN
HM768251,OFFENSE INVOLVING CHILDREN
HT394616,OFFENSE INVOLVING CHILDREN


Question#4: Listing all kidnapping cases involving a child

In [18]:
%sql select * FROM CHICAGO_CRIME_DATA\
WHERE PRIMARY_TYPE="KIDNAPPING" AND DESCRIPTION="CHILD ABDUCTION/STRANGER";

 * sqlite:///FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


QUESTION 5: What types of crimes happened at School?

In [19]:
%sql select PRIMARY_TYPE,LOCATION_DESCRIPTION FROM CHICAGO_CRIME_DATA \
WHERE LOCATION_DESCRIPTION LIKE 'SCHOOL%';

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE,LOCATION_DESCRIPTION
BATTERY,"SCHOOL, PUBLIC, GROUNDS"
BATTERY,"SCHOOL, PUBLIC, BUILDING"
BATTERY,"SCHOOL, PUBLIC, BUILDING"
BATTERY,"SCHOOL, PUBLIC, BUILDING"
BATTERY,"SCHOOL, PUBLIC, GROUNDS"
CRIMINAL DAMAGE,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, BUILDING"
ASSAULT,"SCHOOL, PUBLIC, GROUNDS"
CRIMINAL TRESPASS,"SCHOOL, PUBLIC, GROUNDS"


QUESTION#6: Showing the Average Safety Score for each type of school

In [20]:
%sql select "Elementary, Middle, or High School", AVG(SAFETY_SCORE) FROM CHICAGO_PUBLIC_SCHOOLS_DATA\
GROUP BY "Elementary, Middle, or High School";

 * sqlite:///FinalDB.db
Done.


"Elementary, Middle, or High School",AVG(SAFETY_SCORE)
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


Question #7: Listing the TOP 5 COMMUNITY areas with the highest % of households below the poverty line

In [21]:
%sql  select COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM CENSUS_DATA\
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY desc LIMIT 5;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


Question 8:Finding out which community area is more crime-prone 

In [22]:
%sql select COMMUNITY_AREA_NUMBER, COUNT(CASE_NUMBER) FROM CHICAGO_CRIME_DATA \
GROUP BY COMMUNITY_AREA_NUMBER ORDER BY CASE_NUMBER desc LIMIT 1;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COUNT(CASE_NUMBER)
19.0,8


Question 9: Using a sub-query to find the community with the largest hardship index

In [23]:
%sql select COMMUNITY_AREA_NAME FROM CENSUS_DATA\
WHERE HARDSHIP_INDEX=(select MAX(HARDSHIP_INDEX) FROM CENSUS_DATA);

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Riverdale


Question 10: Creating a sub-query to find the community name with the most crime

In [24]:
%sql select COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME FROM CENSUS_DATA \
WHERE COMMUNITY_AREA_NUMBER=(select COMMUNITY_AREA_NUMBER\
FROM CHICAGO_CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(*) DESC LIMIT 1);

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
25.0,Austin
